# Strands Agents with Bedrock AgentCore Code Interpreter

本研讨会演示如何将 Strands Agents 与 Amazon Bedrock AgentCore Code Interpreter 集成，以创建能够动态执行代码的 AI 代理。

## 概述

在本实验中，您将：
- 了解 Bedrock AgentCore Code Interpreter 功能
- 使用 Strands Agents 测试默认 Code Interpreter
- 创建具有公共网络访问权限的自定义 Code Interpreter

## 先决条件

开始本实验之前，请确保您具备：
- 已配置 AWS 凭证（IAM 角色或环境变量）
- 已安装所需的 Python 包
- 对 Strands Agents 和 Bedrock AgentCore 概念有基本了解

如果您没有在假设 IAM 角色的环境中运行，请将您的 AWS 凭证设置为环境变量：

In [1]:
import os

#os.environ["AWS_ACCESS_KEY_ID"]=<YOUR ACCESS KEY>
#os.environ["AWS_SECRET_ACCESS_KEY"]=<YOUR SECRET KEY>
#os.environ["AWS_SESSION_TOKEN"]=<OPTIONAL - YOUR SESSION TOKEN IF TEMP CREDENTIAL>
#os.environ["AWS_REGION"]=<AWS REGION WITH BEDROCK AGENTCORE AVAILABLE>

为 Strands Agents 和 Bedrock AgentCore Python SDK 安装所需的包：

In [2]:
#%pip install -q strands-agents strands-agents-tools bedrock-agentcore

## 什么是 Bedrock AgentCore Code Interpreter？

Amazon Bedrock AgentCore Code Interpreter 是一个强大的工具，允许 AI 代理在安全的沙盒环境中动态执行代码。主要优势包括：

- **安全执行**：在隔离的沙盒环境中运行 Python 代码
- **动态问题解决**：使代理能够执行计算、分析数据和生成可视化
- **灵活配置**：支持默认沙盒和自定义网络启用环境
- **集成就绪**：与 Strands Agents 和其他 AI 框架无缝集成

Code Interpreter 为代理提供了解决需要计算分析、数据处理或数学计算的复杂问题的能力。

### 使用默认 Code Interpreter 测试 Strands Agent

让我们首先使用默认的 AgentCore Code Interpreter 测试 Strands Agent。我们将演示它如何在沙盒环境中生成和执行代码来解决数学问题。

In [3]:
from strands import Agent
from strands_tools.code_interpreter import AgentCoreCodeInterpreter

agentcore_code_interpreter = AgentCoreCodeInterpreter()

# Create a code gen assistant agent
agent = Agent(
    model="us.amazon.nova-pro-v1:0",  # Optional: Specify the model ID
    system_prompt="""You are a helpful AI assistant that validates all answers through code execution.""",
    tools=[agentcore_code_interpreter.code_interpreter],
)

agent("2+2=")

<thinking> The user's query is a simple arithmetic problem. I can use the code interpreter tool to execute the calculation. </thinking>

Tool #1: code_interpreter
<thinking> It seems the session 'math-session' does not exist. I need to initialize a new session before executing the code. </thinking> 
Tool #2: code_interpreter

Tool #3: code_interpreter
<thinking> The first session initialization was successful, but the second tool call to execute the code failed because the session was not found. I will try executing the code again in the newly created session. </thinking> 
Tool #4: code_interpreter
<thinking> The code execution was successful, and the result of 2 + 2 is 4. I can now provide the answer to the user. </thinking>

The result of 2 + 2 is 4.

AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': '<thinking> The code execution was successful, and the result of 2 + 2 is 4. I can now provide the answer to the user. </thinking>\n\nThe result of 2 + 2 is 4.'}]}, metrics=EventLoopMetrics(cycle_count=4, tool_metrics={'code_interpreter': ToolMetrics(tool={'toolUseId': 'tooluse_kWsHutQLTTGcM8IqGsW49w', 'name': 'code_interpreter', 'input': {'code_interpreter_input': {'action': {'session_name': 'math-session', 'language': 'python', 'code': '2 + 2', 'type': 'executeCode'}}}}, call_count=4, success_count=2, error_count=2, total_time=1.1129376888275146)}, cycle_durations=[1.3124580383300781], traces=[<strands.telemetry.metrics.Trace object at 0xffff575477d0>, <strands.telemetry.metrics.Trace object at 0xffff57c097d0>, <strands.telemetry.metrics.Trace object at 0xffff57550110>, <strands.telemetry.metrics.Trace object at 0xffff57578f10>], accumulated_usage={'inputTokens': 15236, 'outputTokens': 358, 'totalT

让我们检查代理循环的详细执行流程，以了解代理如何处理请求和生成响应：

### 了解沙盒环境限制

默认 Code Interpreter 在具有网络隔离的沙盒环境中运行。这是一个重要的安全功能，可防止未经授权的网络访问，同时确保代码执行安全。

## 创建具有网络访问权限的自定义 Code Interpreter

为了启用基于网络的操作，我们将创建一个具有公共网络访问权限的自定义 Code Interpreter。这展示了 AgentCore 平台在不同用例中的灵活性。

### 步骤 1：初始化 AgentCore 客户端

首先，让我们为控制平面和数据平面操作设置必要的客户端：

In [4]:
from bedrock_agentcore._utils import endpoints
import boto3
import json

region = boto3.session.Session().region_name

data_plane_endpoint = endpoints.get_data_plane_endpoint(region)
control_plane_endpoint = endpoints.get_control_plane_endpoint(region)

cp_client = boto3.client("bedrock-agentcore-control", 
                        region_name=region,
                        endpoint_url=control_plane_endpoint)

dp_client = boto3.client("bedrock-agentcore", 
                        region_name=region,
                        endpoint_url=data_plane_endpoint)

### 步骤 2：创建自定义 Code Interpreter

创建具有公共网络访问权限的自定义 Code Interpreter：

In [5]:
import uuid

# Create code interpreter
unique_name = f"sampleCodeInterpreter_{uuid.uuid4().hex[:8]}"
interpreter_response = cp_client.create_code_interpreter(
    name=unique_name,
    description="Environment for Code Interpreter sample test",
    #executionRoleArn=iam_role_arn, #Required only if the code interpreter need to access AWS resources
    networkConfiguration={
        'networkMode': 'PUBLIC'
    }
)
interpreter_id = interpreter_response["codeInterpreterId"]
print(f"Created interpreter: {interpreter_id}")

Created interpreter: sampleCodeInterpreter_cdab8e01-iD6hT9cHC7


### 步骤 3：创建 Code Interpreter 会话

在自定义 Code Interpreter 中创建代码解释器会话：

In [6]:
# Create code interpreter session
session_response = dp_client.start_code_interpreter_session(
    codeInterpreterIdentifier=interpreter_id,
    name="sampleCodeInterpreterSession",
    sessionTimeoutSeconds=900
)
session_id = session_response["sessionId"]
print(f"Created session: {session_id}")

Created session: 01K3JR0AE1TSEEYWDYDKPZ7Z5A


### 步骤 4：测试基本功能

让我们通过执行简单的 "Hello World" 命令来测试代码解释器会话：

In [7]:
response = dp_client.invoke_code_interpreter(
    codeInterpreterIdentifier=interpreter_id,
    sessionId=session_id,
    name="executeCommand",
    arguments={
        'command': "echo 'Hello World'"
    }
)
for event in response["stream"]:
    print(json.dumps(event["result"], indent=2))

{
  "content": [
    {
      "type": "text",
      "text": "Hello World\r\n"
    }
  ],
  "structuredContent": {
    "stdout": "Hello World\r\n",
    "stderr": "",
    "exitCode": 0,
    "executionTime": 0.005044460296630859
  },
  "isError": false
}


让我们通过使用 pip 安装 Yahoo Finance Python 包及调用 API 查询亚马逊的股价来验证代码解释器会话是否可以访问互联网：

In [8]:
response = dp_client.invoke_code_interpreter(
    codeInterpreterIdentifier=interpreter_id,
    sessionId=session_id,
    name="executeCommand",
    arguments={
        'command': "pip install yfinance"
    }
)

response = dp_client.invoke_code_interpreter(
    codeInterpreterIdentifier=interpreter_id,
    sessionId=session_id,
    name="executeCode",
    arguments={"code": """
        import yfinance as yf
               
        amzn = yf.Ticker('AMZN')
        data = amzn.history(period='1d')
        today_close = data['Close'][-1]
        print(today_close)
        """,
        "language": "python",
        "clearContext": False
    }
)
for event in response["stream"]:
    print(json.dumps(event["result"], indent=2))

{
  "content": [
    {
      "type": "text",
      "text": "227.94000244140625"
    }
  ],
  "structuredContent": {
    "stdout": "227.94000244140625",
    "stderr": "",
    "exitCode": 0,
    "executionTime": 0.9467575550079346
  },
  "isError": false
}


## 在 Strands Agent 中使用自定义 AgentCore Code Interpreter

现在我们将把具有互联网访问功能的 AgentCore Code Interpreter 集成到 Strands Agent 中。我们将
创建自定义的 execute_python 和 execute_command 工具，它们共享同一个代码解释器会
话以增强功能。

In [10]:
from strands import Agent, tool
from bedrock_agentcore.tools.code_interpreter_client import CodeInterpreter
import boto3

#Reuse the Core Interpreter and Session created above
ci_client = CodeInterpreter(region=boto3.session.Session().region_name)
ci_client.start(identifier=interpreter_id) #initializes a new code interpreter session

@tool
def execute_python(code: str, description: str = "") -> str:
    """Execute Python code in the sandbox."""
    
    print(f"\n Generated Code: {code}")
    response = ci_client.invoke("executeCode", {
        "code": code,
        "language": "python",
        "clearContext": False
    })
        
    for event in response["stream"]:
        return json.dumps(event["result"])

@tool
def execute_command(command: str, description: str = "") -> str:
    """Execute command in the sandbox."""
    
    print(f"\n Generated Command: {command}")
    response = ci_client.invoke("executeCommand", {
        "command": command
    })
    
    for event in response["stream"]:
        return json.dumps(event["result"])
    

# Create a code gen agent
agent = Agent(
    model="us.amazon.nova-pro-v1:0", 
    system_prompt="""You are a helpful AI assistant that validates all answers through code execution.
                  If you has no available tools to perform the task, you must generate and execute code to continue. 
                  If required, execute pip install to download required packages. Then retry execute code again after you install package successfully.
                  """,
    tools=[execute_python, execute_command],
)

agent("今天亚马逊的股价是多少？")

ci_client.stop() #stop the current code interpreter session

<thinking> To get the current stock price of Amazon, I need to use a financial data API. However, I do not have direct access to external financial data APIs. I will need to use a Python package that can fetch this information. The `yfinance` package is commonly used for this purpose. I will first check if the package is installed and then proceed to fetch the stock price. </thinking>


Tool #1: execute_python

 Generated Code: import yfinance as yf

# Fetch the current stock price of Amazon
amzn = yf.Ticker('AMZN')
data = amzn.history(period='1d')
current_price = data['Close'][0]
current_price
<thinking> The `yfinance` package is not installed, so I need to install it first. After installation, I will retry fetching the stock price. </thinking> 
Tool #2: execute_command

 Generated Command: pip install yfinance
<thinking> The `yfinance` package has been successfully installed. I will now retry fetching the current stock price of Amazon. </thinking> 
Tool #3: execute_python

 Generated

## 资源清理（可选）

清理我们创建的 AgentCore Runtime 资源以避免不必要的费用：

In [11]:
import boto3

region = boto3.session.Session().region_name

cp_client = boto3.client("bedrock-agentcore-control", region_name=region, endpoint_url=control_plane_endpoint)
dp_client = boto3.client("bedrock-agentcore", region_name=region, endpoint_url=data_plane_endpoint)

try:
    print("Cleaning up session and interpreter...")
    dp_client.stop_code_interpreter_session(
        codeInterpreterIdentifier=interpreter_id,
        sessionId=session_id
    )
    print("✓ Session stopped successfully")

    cp_client.delete_code_interpreter(codeInterpreterId=interpreter_id)
    print("✓ Interpreter deleted successfully")
except Exception as e:
    print(f"❌ Error during cleanup: {e}")
    print("You may need to manually clean up some resources.")

Cleaning up session and interpreter...
✓ Session stopped successfully
✓ Interpreter deleted successfully
